In [132]:
import pandas as pd
import numpy as np
import string
from collections import Counter
from nltk.corpus import stopwords

In [139]:
train_dataset=pd.DataFrame(pd.read_csv('train.csv'))
dev_dataset=pd.read_csv('dev.csv')
train_dataset.head()
stopWords = stopwords.words("english")

In [140]:
#Clean data, remove punctuation and lower the text
count=0
def clean_data(x):
    #print('Running cleaning')
    x=x['Comment'].lower()
    x=x.translate(x.maketrans(string.punctuation,' ' * len(string.punctuation)))
    return x.replace('  ',' ').strip()


def vectorize2(x):
    tokens=Counter(x['Comment'].split())
    for item in words:
        x[item]=0
        if tokens[item]>0:
            x[item]=tokens[item]
    return x


def vectorize(x):
    tokens=[word for word in x['Comment'].split() if word not in stopWords]
    for word in tokens:
        #print(words_df)
        if x['Insult']==1:
            words_df[word][1]=words_df[word][1]+1
        if x['Insult']==0:
            words_df[word][0]=words_df[word][0]+1
    return words_df


In [141]:
#Use Bernoulli

#Clean data
train_dataset['Comment']=train_dataset.apply(clean_data, axis=1)
#train_dataset.head()
#Create a set of all the words(except stop words) occurring in the comments
words=set()

#Dropping stop words
train_dataset.apply(lambda x: words.update(word for word in x['Comment'].split() if word not in stopWords),axis=1)


#Removing this word since it's a part of the column


print('Total distinct words in the comment is',len(words))
#train_dataset

words_df=pd.DataFrame(index=range(0,2),columns=words).fillna(0)
#Index 0 for inuslt 0, and index 1 for insult 1
#train_dataset
train_dataset.apply(vectorize, axis=1)
words_df

Total distinct words in the comment is 16336


,comparing,popular,berry,ovonic,viet,daddy,nsolid,ricany,nmanu,roosevelt,...,portugal,stranges,approvel,x83,contributor,applies,gordon,nforget,nl,putnam
0,3,1,2,1,1,8,1,1,1,1,...,1,1,1,1,3,1,1,1,1,1
1,1,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
#Aggregations

total=words_df.sum(axis=1)
tot_0=total[0]
tot_1=total[1]
totalwords=tot_1+tot_0

print('Count of 1\'s=',tot_1,',and Count of 0\'s is ',tot_0,',Total Count is ',totalwords)
prob_0=tot_0/totalwords #Probabilty of 0, ie. not insult
prob_1=tot_1/totalwords #Probabilty of 1, ie. insult
print('Probability of 1\'s=',prob_1,',and Probability of 0\'s is ',prob_0)

#Counting numbers of words in each class for smoothing
words_count=words_df.astype(bool).sum(axis=1)
wordsin0=words_count[0]
wordsin1=words_count[1]

#word_prob_gvn0=words_df.apply(lambda x: x[0]/tot_0,axis=0)
#word_prob_gvn1=words_df.apply(lambda x: x[1]/tot_1,axis=0)
#Using smoothning
word_prob_gvn0=words_df.apply(lambda x: (x[0]+1)/(tot_0+wordsin0),axis=0)
word_prob_gvn1=words_df.apply(lambda x: (x[1]+1)/(tot_1+wordsin1),axis=0)


#word_prob_tot=words_df.apply(lambda x: (x[1]+x[0])/totalwords,axis=0)

prob_matrix=pd.concat([word_prob_gvn0,word_prob_gvn1],axis=1).T
#Saving probability matrix, 
#row 1 is prob of word given 0, 
# 2nd is prob of word given 1
# 3rd is prob of word in total
prob_matrix.to_csv('prob_out.csv',index=False)
prob_matrix

Count of 1's= 15088 ,and Count of 0's is  61233 ,Total Count is  76321
Probability of 1's= 0.19769133004022485 ,and Probability of 0's is  0.8023086699597751


,comparing,popular,berry,ovonic,viet,daddy,nsolid,ricany,nmanu,roosevelt,...,portugal,stranges,approvel,x83,contributor,applies,gordon,nforget,nl,putnam
0,0.000053,0.000026,0.00004,0.000026,0.000026,0.000119,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000026,0.000026,0.000026,0.000053,0.000026,0.000026,0.000026,0.000026,0.000026
1,0.000100,0.000050,0.00005,0.000050,0.000050,0.000300,0.000050,0.000050,0.000050,0.000050,...,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050


In [193]:
#P(1|Word)=(P(Word|1)*P(1))/P(Word)
#P(1|Words)=P(word1|1)*P(word2|1)*P(word3|1)...*P(1)/(P(word1)*P(word2)*P(word3)...)
# Prob of word1, word2 ca be ignored when comparing probability
#probability of 1, given words=((probability of words given 1)*probability of words)/(probabilty of words)

In [169]:
prob_out=pd.read_csv('prob_out.csv')
prob_out

#row 1 is prob of word given 0, 
# 2nd is prob of word given 1

,comparing,popular,berry,ovonic,viet,daddy,nsolid,ricany,nmanu,roosevelt,...,portugal,stranges,approvel,x83,contributor,applies,gordon,nforget,nl,putnam
0,0.000053,0.000026,0.00004,0.000026,0.000026,0.000119,0.000026,0.000026,0.000026,0.000026,...,0.000026,0.000026,0.000026,0.000026,0.000053,0.000026,0.000026,0.000026,0.000026,0.000026
1,0.000100,0.000050,0.00005,0.000050,0.000050,0.000300,0.000050,0.000050,0.000050,0.000050,...,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050


In [170]:
#Pass df of comment

#It will first clean the comment, Split the comment and remove stop words
# Get the probability of that word when 1 is given and net probability of that word
#Use Bayes' theorem to calculate req probability

def test_data(x):
    comment=clean_data(x)
    net_prob1=float(1.0)
    net_prob0=float(1.0)
    
    words=[word for word in comment.split() if word not in stopWords]
    
    for word in words:
        if word not in prob_out.columns:
            net_prob0=net_prob0*(1/(tot_0+wordsin0))
            net_prob1=net_prob1*(1/(tot_1+wordsin1))
        else:
            wordprob_gvn0=prob_out[word][0]
            wordprob_gvn1=prob_out[word][1]
            net_prob0=(net_prob0*wordprob_gvn0)
            net_prob1=(net_prob1*wordprob_gvn1)

    prob0_gvnwords=net_prob0*prob_0
    prob1_gvnwords=net_prob1*prob_1
    return 1 if prob1_gvnwords> prob0_gvnwords else 0
    #return prob0_gvnwords,prob1_gvnwords

#Output code

In [191]:
#

dev_dataset=pd.read_csv('dev.csv')
#dev_dataset.head()
dev_dataset['predicted']=dev_dataset.apply(test_data,axis=1)
correct=dev_dataset[dev_dataset['Insult']==dev_dataset['predicted']].shape[0]
incorrect=dev_dataset[dev_dataset['Insult']!=dev_dataset['predicted']].shape[0]
print('Correct:', correct,', Incorrect:',incorrect,'correct fraction',correct/(correct+incorrect))

#dev_dataset[dev_dataset['Insult']!=dev_dataset['predicted']]

Correct: 1462 , Incorrect: 773 correct fraction 0.6541387024608502


In [186]:
#Doubts
#How to make model ot of it?
# Is it just like method call?
# What if input has same word multiple times?



#print('Word:',word,' Prob:',prob_out[word])
        #print('Word:',word,' Prob0:',prob_out[word][0])
        #print('Word is:',word,', wordprob_gvn0:',wordprob_gvn0)
        #print('wordprob_gvn1: ',wordprob_gvn1,', prob_word:',prob_word)
        
        
x='"You are an opinionated, uneducated idiot. And Kingdom of Heaven Director\'s Cut is a masterpiece."'
x=x.translate(x.maketrans(string.punctuation,' ' * len(string.punctuation)))
x=x.replace('  ',' ').strip().lower()

probs=test_data(x)
print('Probability of 0 is: ',probs[0],' and probability of 1 is: ',probs[1])